In [32]:
from dotenv import load_dotenv
load_dotenv()

from src.loader import load_pdfs
from src.cleaner import clean_documents
from src.splitter import split_docs
from src.embeddings import get_embeddings
from src.vectorstore import build_vectorstore
from src.qa import create_qa_chain

In [33]:
docs=load_pdfs()

Loaded 2348 documents


In [34]:
cleaned_docs = clean_documents(docs)

Cleaned 2348 documents


In [35]:
chunks = split_docs(cleaned_docs)

Created 8360 chunks


In [36]:
embeddings = get_embeddings()

Using sentence-transformers/all-MiniLM-L6-v2 on cpu


In [37]:
vector_db = build_vectorstore(chunks, embeddings)

Vector DB persisted at chroma_db


In [38]:
qa_chain = create_qa_chain(vector_db)

RetrievalQA chain ready using llama3-70b-8192 and LangSmith project 'b-arch-chatbot'


In [39]:
query = "Aluminium cladding has a similar range of profiles"
result = qa_chain.invoke({"query": query})

print("Answer:", result["result"])
print("Sources:", [doc.metadata for doc in result["source_documents"]])

Answer: Based on the provided context, it can be inferred that metal cladding profiles serve as a general category, encompassing various types of cladding materials, including aluminium. 

As the context specifically mentions "metal cladding profiles," it is reasonable to assume that aluminium cladding, being a type of metal cladding, would fall under this category. 

Therefore, it can be logically deduced that aluminium cladding would have a similar range of profiles as metal cladding profiles in general. 

This conclusion is supported by the categorical relationship between metal cladding profiles and aluminium cladding, where the latter is a subset of the former.

(Source: Context - Metal cladding profiles)

Please note that this answer is based on the limited information provided and may require further evidence or research to establish a more comprehensive understanding of the topic.
Sources: [{'page': 39, 'author': 'shafnaibm@gmail.com', 'trapped': '', 'total_pages': 58, 'format'